In [1]:
import cobra
from cobra.io import load_json_model
import numpy as np
import pandas as pd
import ray
cobra_model = load_json_model("./COBRA_models/GEM_Recon2_thermocurated_redHUMAN.json")
import itertools

def grow_dict(previo, nuevo):  
    """Esto es un .update(overwritte=False)?"""

    not_new_keys = set(previo.keys()).intersection(set(nuevo.keys()))
    new_keys     = set(nuevo.keys()).difference(set(previo.keys()))
    for new in new_keys:
        previo[new] =np.unique(nuevo[new]).tolist() 
    for common in not_new_keys:
        previo[common].extend(np.unique(nuevo[common]).tolist() )
    return previo
# Sacamos los genes de Recon hacia un diccionario
# TODO: convertir est en una horrible dict_comprehension { curse : cursed for c in curseds }
dict_rxn_to_gene = dict()
genes            = [g.id for g in cobra_model.genes]

#print(genes)


for id in genes:
    # fmt: off
    rxns             = [r.id for r in cobra_model.genes.get_by_id(id)._reaction] # reacciones asociadas al gen de la iter
    updated_dict  : dict[str, list[str]] = dict(zip(rxns,itertools.repeat([id]))) # zipping con la { reaccion : [id_gen] }
    dict_rxn_to_gene = grow_dict(dict_rxn_to_gene, updated_dict) # merge recursivo

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [2]:
dict_rxn_to_gene["METLEUex"]

['6520.1', '8140.1']

In [3]:
edges_node_names_explainer_subgraphs = pd.read_csv("./results/data/data_for_R/Non_masked_edges_node_names_explainer_subgraphs.csv").drop("Unnamed: 0",axis=1)


edges_node_names_explainer_subgraphs

,node1,node2,0,1,2,3,4,5,6,7,...,4911,4912,4913,4914,4915,4916,4917,4918,4919,4920
0,r1352,phe_L_c,0.014385,0.011012,0.013806,0.013806,0.037142,0.061166,0.013870,0.026582,...,0.013782,0.073532,0.013829,0.683351,0.013883,0.091757,0.139979,0.081921,0.013657,0.030380
1,r1341,phe_L_c,0.014653,0.010811,0.013767,0.013774,0.041098,0.070738,0.013744,0.027853,...,0.013798,0.082251,0.013729,0.762316,0.013711,0.104231,0.167696,0.093851,0.013793,0.032845
2,r1344,phe_L_c,0.014533,0.010893,0.013741,0.013772,0.038688,0.065435,0.013778,0.027111,...,0.013889,0.078924,0.013881,0.722953,0.013835,0.099442,0.153985,0.088249,0.013745,0.031529
3,r1763,phe_L_e,0.697588,0.147041,0.013848,0.013737,0.684845,0.727760,0.013833,0.697106,...,0.013805,0.692684,0.013784,0.915297,0.013892,0.726709,0.745279,0.725146,0.013715,0.681900
4,r1763,phe_L_c,0.020129,0.019571,0.013831,0.013778,0.031661,0.033554,0.013887,0.029057,...,0.013721,0.051231,0.013626,0.693623,0.013768,0.076526,0.077942,0.054164,0.013843,0.028267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,r1345,phe_L_c,0.014415,0.011178,0.013807,0.013675,0.036387,0.058408,0.013740,0.025984,...,0.013823,0.070975,0.013917,0.614360,0.013725,0.086485,0.132335,0.077464,0.013664,0.030141
381,r1813,phe_L_c,0.020551,0.022101,0.013686,0.013782,0.042798,0.064855,0.013804,0.062404,...,0.013743,0.082138,0.013771,0.347872,0.013876,0.086213,0.461182,0.074207,0.013698,0.067360
382,r1813,phe_L_e,0.678947,0.540139,0.013634,0.013885,0.715388,0.718994,0.013858,0.748870,...,0.013747,0.696515,0.013711,0.870952,0.013736,0.703119,0.762884,0.687474,0.013762,0.722276
383,r1562,phe_L_e,0.063387,0.064093,0.013764,0.013916,0.375260,0.430964,0.013689,0.496722,...,0.013880,0.554562,0.013854,0.900793,0.013786,0.549427,0.653751,0.573836,0.013887,0.340239


In [4]:

ray.shutdown()
ray.init()
@ray.remote


def get_genes(node_1, node_2):
    
    if node_2 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_2]
        
    elif node_1 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_1]
        
    else:
        return ['']
    
g = [] 
for node_1, node_2 in zip(edges_node_names_explainer_subgraphs.node1.tolist(), edges_node_names_explainer_subgraphs.node2.tolist()):   
    g.append(get_genes.remote(node_1, node_2 ))

 
genes = pd.Series(ray.get(g) , name='genes')



assert genes.shape[0] == edges_node_names_explainer_subgraphs.shape[0]


2022-11-05 18:46:41,435	INFO worker.py:1518 -- Started a local Ray instance.


In [5]:
results_for_R = pd.concat([genes, edges_node_names_explainer_subgraphs], axis=1)

results_for_R

,genes,node1,node2,0,1,2,3,4,5,6,...,4911,4912,4913,4914,4915,4916,4917,4918,4919,4920
0,[],r1352,phe_L_c,0.014385,0.011012,0.013806,0.013806,0.037142,0.061166,0.013870,...,0.013782,0.073532,0.013829,0.683351,0.013883,0.091757,0.139979,0.081921,0.013657,0.030380
1,[],r1341,phe_L_c,0.014653,0.010811,0.013767,0.013774,0.041098,0.070738,0.013744,...,0.013798,0.082251,0.013729,0.762316,0.013711,0.104231,0.167696,0.093851,0.013793,0.032845
2,[],r1344,phe_L_c,0.014533,0.010893,0.013741,0.013772,0.038688,0.065435,0.013778,...,0.013889,0.078924,0.013881,0.722953,0.013835,0.099442,0.153985,0.088249,0.013745,0.031529
3,[9057.1],r1763,phe_L_e,0.697588,0.147041,0.013848,0.013737,0.684845,0.727760,0.013833,...,0.013805,0.692684,0.013784,0.915297,0.013892,0.726709,0.745279,0.725146,0.013715,0.681900
4,[9057.1],r1763,phe_L_c,0.020129,0.019571,0.013831,0.013778,0.031661,0.033554,0.013887,...,0.013721,0.051231,0.013626,0.693623,0.013768,0.076526,0.077942,0.054164,0.013843,0.028267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,[],r1345,phe_L_c,0.014415,0.011178,0.013807,0.013675,0.036387,0.058408,0.013740,...,0.013823,0.070975,0.013917,0.614360,0.013725,0.086485,0.132335,0.077464,0.013664,0.030141
381,[9057.1],r1813,phe_L_c,0.020551,0.022101,0.013686,0.013782,0.042798,0.064855,0.013804,...,0.013743,0.082138,0.013771,0.347872,0.013876,0.086213,0.461182,0.074207,0.013698,0.067360
382,[9057.1],r1813,phe_L_e,0.678947,0.540139,0.013634,0.013885,0.715388,0.718994,0.013858,...,0.013747,0.696515,0.013711,0.870952,0.013736,0.703119,0.762884,0.687474,0.013762,0.722276
383,[8140.1],r1562,phe_L_e,0.063387,0.064093,0.013764,0.013916,0.375260,0.430964,0.013689,...,0.013880,0.554562,0.013854,0.900793,0.013786,0.549427,0.653751,0.573836,0.013887,0.340239


In [6]:
results_for_R.to_csv("./results/data/data_for_R/Non_masked_results_for_R.csv")

# Masked

In [7]:
import cobra
from cobra.io import load_json_model
import numpy as np
import pandas as pd
import ray
cobra_model = load_json_model("./COBRA_models/GEM_Recon2_thermocurated_redHUMAN.json")
import itertools

def grow_dict(previo, nuevo):  
    """Esto es un .update(overwritte=False)?"""

    not_new_keys = set(previo.keys()).intersection(set(nuevo.keys()))
    new_keys     = set(nuevo.keys()).difference(set(previo.keys()))
    for new in new_keys:
        previo[new] =np.unique(nuevo[new]).tolist() 
    for common in not_new_keys:
        previo[common].extend(np.unique(nuevo[common]).tolist() )
    return previo
# Sacamos los genes de Recon hacia un diccionario
# TODO: convertir est en una horrible dict_comprehension { curse : cursed for c in curseds }
dict_rxn_to_gene = dict()
genes            = [g.id for g in cobra_model.genes]

#print(genes)


for id in genes:
    # fmt: off
    rxns             = [r.id for r in cobra_model.genes.get_by_id(id)._reaction] # reacciones asociadas al gen de la iter
    updated_dict  : dict[str, list[str]] = dict(zip(rxns,itertools.repeat([id]))) # zipping con la { reaccion : [id_gen] }
    dict_rxn_to_gene = grow_dict(dict_rxn_to_gene, updated_dict) # merge recursivo
    
edges_node_names_explainer_subgraphs = pd.read_csv("./results/data/data_for_R/Masked_edges_node_names_explainer_subgraphs.csv").drop("Unnamed: 0",axis=1)
ray.shutdown()
ray.init()
@ray.remote


def get_genes(node_1, node_2):
    
    if node_2 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_2]
        
    elif node_1 in dict_rxn_to_gene.keys():
        
        return dict_rxn_to_gene[node_1]
        
    else:
        return ['']
    
g = [] 
for node_1, node_2 in zip(edges_node_names_explainer_subgraphs.node1.tolist(), edges_node_names_explainer_subgraphs.node2.tolist()):   
    g.append(get_genes.remote(node_1, node_2 ))

 
genes = pd.Series(ray.get(g) , name='genes')



assert genes.shape[0] == edges_node_names_explainer_subgraphs.shape[0]



2022-11-05 18:47:07,343	INFO worker.py:1518 -- Started a local Ray instance.


In [9]:
results_for_R = pd.concat([genes, edges_node_names_explainer_subgraphs], axis=1)

results_for_R

,genes,node1,node2,0,1,2,3,4,5,6,...,4911,4912,4913,4914,4915,4916,4917,4918,4919,4920
0,[1719.1],r0514,fol_m,0.031795,0.444939,0.015893,0.014116,0.031279,0.476097,0.436362,...,0.494638,0.033656,0.026609,0.022163,0.107112,0.014560,0.036056,0.017274,0.423704,0.393771
1,[1806.1],DURAD2,56dthm_c,0.026460,0.504451,0.013868,0.446239,0.055073,0.438663,0.460749,...,0.481807,0.019728,0.018303,0.013193,0.017885,0.426556,0.016502,0.511591,0.472736,0.450847
2,[1384.2],CSNATp,acrn_x,0.286803,0.290118,0.397468,0.182305,0.257072,0.254167,0.271842,...,0.346363,0.388449,0.290275,0.386786,0.260111,0.089285,0.290353,0.245498,0.321590,0.194852
3,"[6120.1, 6120.2]",RPE,xu5p_D_c,0.703749,0.016609,0.017448,0.016211,0.013607,0.019640,0.019261,...,0.016130,0.568603,0.658818,0.019113,0.016333,0.017437,0.581916,0.016556,0.021154,0.023291
4,"[4846.1, 201288.1, 4842.1, 4843.2, 4843.1, 645...",NOS2,nwharg_c,0.049078,0.377641,0.506255,0.466333,0.332245,0.294191,0.417322,...,0.363098,0.025704,0.098717,0.480220,0.764428,0.375065,0.020703,0.409896,0.395963,0.434109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,"[6510.1, 6509.1]",ALATHRNaEx,ala_L_c,0.025539,0.188538,0.486464,0.470900,0.025536,0.086797,0.039834,...,0.064332,0.027269,0.029209,0.020487,0.020365,0.531651,0.098338,0.239677,0.208890,0.128658
381,"[112724.1, 145226.1, 51109.1, 195814.1, 57665.1]",retinal_11_cis_c,RDH3a,0.237799,0.014213,0.014290,0.014361,0.236460,0.014714,0.014528,...,0.014422,0.151541,0.239490,0.014467,0.236248,0.014432,0.252864,0.014498,0.014836,0.014547
382,[8418.1],HC01162_c,r0267,0.223937,0.014489,0.014628,0.014595,0.230775,0.014863,0.014847,...,0.014677,0.137456,0.234180,0.014573,0.245183,0.014795,0.234434,0.014584,0.015215,0.014667
383,[10991.1],r1143,asp_L_c,0.168699,0.117035,0.169650,0.114702,0.206386,0.213727,0.015484,...,0.238673,0.043295,0.365543,0.174049,0.226757,0.483136,0.031987,0.047714,0.068043,0.234095


In [10]:
results_for_R.to_csv("./results/data/data_for_R/Masked_results_for_R.csv")